# 要約 
このJupyterノートブックは、Kaggleコンペティション「LMSYS - Chatbot Arena」における人間による好みの予測を行うためのモデルを構築することを目的としています。具体的には、ユーザーが生成したプロンプトに対する2つの異なる応答から、どちらがより好まれるかを予測します。

ノートブックでは、以下の手法とライブラリが活用されています。

1. **ライブラリのインストールおよびインポート**：
   - `bitsandbytes`、`transformers`、`tokenizers`、`peft`など、モデルのトレーニングや推論を効率化するためのライブラリを使用しています。
   - PyTorch（`torch`）、scikit-learn（`sklearn`）、pandas（`pd`）、numpy（`np`）などの一般的なデータ処理および機械学習ライブラリもインポートされています。

2. **データ処理**：
   - テストデータを読み込み、プロンプトやレスポンスの前処理を行います。具体的には、null値の処理やトークン化を行い、モデルの入力として使用するフォーマットに整えています。

3. **トークナイゼーションとモデルの設定**：
   - Hugging FaceのTransformersを利用して、`Gemma2ForSequenceClassification`モデルを使用し、トークナイザーを設定しています。これにより、テキストデータをモデルが理解できる形式に変換します。

4. **推論の実施**：
   - 2つの異なるGPUを用いて並行処理を行い、それぞれのモデルからの出力や確率を計算します。これにより、各モデルがどの程度の確率でユーザーの好みを予測するかを評価します。

5. **特徴量エンジニアリングとLightGBMの統合**：
   - `CountVectorizer`を利用して特徴量を抽出し、LightGBMを使った予測も組み合わせています。これにより、異なるアルゴリズムを統合した予測を可能にしています。

6. **結果のブレンド**：
   - LightGBMによる予測とLLMの予測を重み付けしてブレンドし、最終的な出力を生成します。この出力は、ユーザーの好みと応答の優劣を示す確率として出されます。

最終的に、生成された予測結果はCSVファイルに保存され、Kaggleに提出するための形式に整えられます。このノートブックは、複数の技術やモデルを駆使しながら、ユーザーの好みをより正確に推測するための包括的なアプローチを示しています。

---


# 用語概説 
以下は、Jupyter Notebookの中で初心者がつまずきそうな専門用語の簡単な解説です。特にマイナーな用語や実務経験がなければ馴染みが薄いと思われるものに焦点を当てています。

### 専門用語の解説

1. **bitsandbytes**:
   - メモリ効率の良いオペレーションを提供するライブラリ。特に大規模なモデルを扱う際のメモリ管理や演算を最適化するために使用される。

2. **tokenizers**:
   - 自然言語処理でテキストをトークン化（語や文を分割）するためのライブラリ。トークン化は、機械学習モデルが理解できる形式にテキストを変換するために重要。

3. **PEFT (Parameter-Efficient Fine-Tuning)**:
   - モデルの微調整手法の一つ。より少ないパラメータを更新して、性能を最大化することを目指す手法で、特にメモリ使用効率を高めるために利用される。

4. **LoraConfig**:
   - LoRA（Low-Rank Adaptation）に関する設定。深層学習モデルをより効率的に訓練するためのテクニックで、モデルの一部のパラメータを低ランクの形式で調整する手法。

5. **mixed precision (自動混合精度)**:
   - 計算の精度を動的に調整する手法。通常、モデルの訓練を効率化し、GPUのメモリ使用量を削減するために使用されます。

6. **attention mask**:
   - トークン化された入力に対して、どのトークンが意味を持ち（注意を払われるべき）どのトークンが無視できるかを示すマスク。トランスフォーマーモデルで入力の効果的な処理を助ける。

7. **pad_without_fast_tokenizer_warning**:
   - 自動的に入力をパディング（同じ長さにすること）する関数であり、トークンの数が異なる入力を処理する際に使われます。この関数は、パディングに関する警告を避けるために設計されています。

8. **symlog (シンメトリックログ変換)**:
   - 数値の符号に応じて異なる変換を施す方法。正の値は通常のログ変換、負の値にはマイナスの符号を付けることで、データ分布を整えるために用います。

9. **dense (疎行列をDense行列に変換)**:
   - 疎行列（多くのゼロ要素を持つ行列）を密行列（ゼロ以外の要素が多数ある行列）に変換すること。計算の効率性向上に寄与します。

10. **StratifiedKFold**:
    - 層化K-Foldクロスバリデーションを行うためのクラス。データを異なるカテゴリーに均等に分割して、よりバランスの取れた訓練と検証を行う方法。

これらの用語は、実務に慣れていないと留意しないかもしれないものですが、特に深層学習や自然言語処理の分野においては非常に重要です。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
<div class="column-right">

# 日本語訳

```python
# bitsandbytesをインストールします。これは、メモリ効率の良いモデルやオペレーションを提供するライブラリです。
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
# transformersライブラリをインストールします。これは、さまざまな事前学習済みモデルを使用するためのライブラリです。
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
# tokenizersライブラリをインストールします。これは、トークン化処理を効率的に行うためのライブラリです。
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
# peftライブラリをインストールします。これは、学習のためのフレームワークであり、通常はメモリの使用効率を改善するために使用されます。
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
</details>

In [ ]:
# bitsandbytesをインストールします。これは、メモリ効率の良いモデルやオペレーションを提供するライブラリです。
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
# transformersライブラリをインストールします。これは、さまざまな事前学習済みモデルを使用するためのライブラリです。
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
# tokenizersライブラリをインストールします。これは、トークン化処理を効率的に行うためのライブラリです。
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
# peftライブラリをインストールします。これは、学習のためのフレームワークであり、通常はメモリの使用効率を改善するために使用されます。
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files
```

</div>
<div class="column-right">

# 日本語訳

```python
# transformers、peft、accelerate、bitsandbytesの各ライブラリをインストールします。
# これらは、モデルを簡単に使用するための重要なライブラリです。
# インストールは最新バージョンにアップグレードし、特定のローカルリンクから行います。
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files
```

</div>
</details>

In [ ]:
# transformers、peft、accelerate、bitsandbytesの各ライブラリをインストールします。
# これらは、モデルを簡単に使用するための重要なライブラリです。
# インストールは最新バージョンにアップグレードし、特定のローカルリンクから行います。
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

The work in this notebook is inspired by these notebooks:
* https://www.kaggle.com/code/ivanvybornov/llama3-8b-lgbm-tfidf
* https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b

</div>
<div class="column-right">

# 日本語訳

このノートブックの作業は、以下のノートブックに触発されています。
* https://www.kaggle.com/code/ivanvybornov/llama3-8b-lgbm-tfidf
* https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel
import os
```

</div>
<div class="column-right">

# 日本語訳

```python
# 必要なライブラリをインポートします。
# timeライブラリは、時間の計測を行うために使用します。
import time
# dataclassを使用することで、簡単にデータ構造を定義できます。
from dataclasses import dataclass
# ThreadPoolExecutorを使って、スレッドプールを管理し並行処理を実行します。
from concurrent.futures import ThreadPoolExecutor

# torchライブラリは、PyTorchフレームワークの主要な部分で、テンソル計算を行います。
import torch
# sklearnライブラリは、機械学習のためのツールセットを提供します。
import sklearn
# numpyは、高速な数値計算のためのライブラリです。
import numpy as np
# pandasは、データ操作と解析のためのライブラリです。
import pandas as pd
# Gemma2ForSequenceClassificationは、テキストの分類を実行するためのモデルです。
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
# データコラトを使用して、データを適切な形に整形します。
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
# PeftModelは、モデルの微調整用のライブラリです。
from peft import PeftModel
# osライブラリは、オペレーティングシステムとのインターフェースを提供します。
import os
```

</div>
</details>

In [ ]:
# 必要なライブラリをインポートします。
# timeライブラリは、時間の計測を行うために使用します。
import time
# dataclassを使用することで、簡単にデータ構造を定義できます。
from dataclasses import dataclass
# ThreadPoolExecutorを使って、スレッドプールを管理し並行処理を実行します。
from concurrent.futures import ThreadPoolExecutor

# torchライブラリは、PyTorchフレームワークの主要な部分で、テンソル計算を行います。
import torch
# sklearnライブラリは、機械学習のためのツールセットを提供します。
import sklearn
# numpyは、高速な数値計算のためのライブラリです。
import numpy as np
# pandasは、データ操作と解析のためのライブラリです。
import pandas as pd
# Gemma2ForSequenceClassificationは、テキストの分類を実行するためのモデルです。
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
# データコラトを使用して、データを適切な形に整形します。
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
# PeftModelは、モデルの微調整用のライブラリです。
from peft import PeftModel
# osライブラリは、オペレーティングシステムとのインターフェースを提供します。
import os

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from threading import Thread
import gc
import os
import io
import json
import random
import pickle
import zipfile
import datetime
import time

import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from torch.cuda.amp import autocast
from IPython.display import display
import torch.nn.functional as F
import tokenizers
```

</div>
<div class="column-right">

# 日本語訳

```python
# さらに必要なライブラリをインポートします。
# Threadを使って、独立したスレッドを作成して並列処理を行います。
from threading import Thread
# gcライブラリでは、ガベージコレクションを手動で管理できます。
import gc
# osライブラリは、オペレーティングシステムとのファイルシステムにアクセスするための関数を提供します。
import os
# ioライブラリは、入出力操作をサポートします。
import io
# jsonライブラリは、JSONデータを扱うための機能を提供します。
import json
# randomライブラリは、ランダム数の生成を行います。
import random
# pickleライブラリは、Pythonオブジェクトのシリアライズ（保存）とデシリアライズ（読み込み）を行います。
import pickle
# zipfileライブラリは、ZIPファイルの読み書きをサポートします。
import zipfile
# datetimeライブラリは、日付と時刻を扱うための機能を提供します。
import datetime
# timeライブラリは、時間に関する操作を行います。
import time

# torchライブラリは、PyTorchフレームワークの主要な部分で、テンソル計算を行います。
import torch
# numpyは、高速な数値計算のためのライブラリです。
import numpy as np
# pandasは、データ操作と解析のためのライブラリです。
import pandas as pd
# AutoTokenizerは、様々なモデルのトークナイザーを自動的に読み込むことができます。
# LlamaModelおよびLlamaForSequenceClassificationは、Llamaモデルとその分類用のフレームワークです。
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
# PEFT関連のモデルと設定を管理するためのライブラリです。
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
# CUDAの自動キャストを使用して、計算の精度を調整します。
from torch.cuda.amp import autocast
# IPython.displayを使用して、Jupyter Notebook上での表示操作を行います。
from IPython.display import display
# Fは、PyTorchの関数を含むモジュールで、特に損失関数や活性化関数を利用するために使います。
import torch.nn.functional as F
# tokenizersライブラリは、トークン化を効率的に行うためのライブラリです。
import tokenizers
```

</div>
</details>

In [ ]:
# さらに必要なライブラリをインポートします。
# Threadを使って、独立したスレッドを作成して並列処理を行います。
from threading import Thread
# gcライブラリでは、ガベージコレクションを手動で管理できます。
import gc
# osライブラリは、オペレーティングシステムとのファイルシステムにアクセスするための関数を提供します。
import os
# ioライブラリは、入出力操作をサポートします。
import io
# jsonライブラリは、JSONデータを扱うための機能を提供します。
import json
# randomライブラリは、ランダム数の生成を行います。
import random
# pickleライブラリは、Pythonオブジェクトのシリアライズ（保存）とデシリアライズ（読み込み）を行います。
import pickle
# zipfileライブラリは、ZIPファイルの読み書きをサポートします。
import zipfile
# datetimeライブラリは、日付と時刻を扱うための機能を提供します。
import datetime
# timeライブラリは、時間に関する操作を行います。
import time

# torchライブラリは、PyTorchフレームワークの主要な部分で、テンソル計算を行います。
import torch
# numpyは、高速な数値計算のためのライブラリです。
import numpy as np
# pandasは、データ操作と解析のためのライブラリです。
import pandas as pd
# AutoTokenizerは、様々なモデルのトークナイザーを自動的に読み込むことができます。
# LlamaModelおよびLlamaForSequenceClassificationは、Llamaモデルとその分類用のフレームワークです。
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
# PEFT関連のモデルと設定を管理するためのライブラリです。
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
# CUDAの自動キャストを使用して、計算の精度を調整します。
from torch.cuda.amp import autocast
# IPython.displayを使用して、Jupyter Notebook上での表示操作を行います。
from IPython.display import display
# Fは、PyTorchの関数を含むモジュールで、特に損失関数や活性化関数を利用するために使います。
import torch.nn.functional as F
# tokenizersライブラリは、トークン化を効率的に行うためのライブラリです。
import tokenizers

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
assert torch.cuda.device_count() == 2

```

</div>
<div class="column-right">

# 日本語訳

```python
# 使用可能なCUDAデバイスの数が2であることを確認します。
# これは、GPUが2つある場合に正しく動作することを保証します。
assert torch.cuda.device_count() == 2
```

</div>
</details>

In [ ]:
# 使用可能なCUDAデバイスの数が2であることを確認します。
# これは、GPUが2つある場合に正しく動作することを保証します。
assert torch.cuda.device_count() == 2

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# infer

</div>
<div class="column-right">

# 日本語訳

# 推論

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5600'
    max_length = 2048
    batch_size = 4
    device = torch.device("cuda")    
    tta = False  # test time augmentation. <prompt>-<model-b's response>-<model-a's response>
    spread_max_length = False  # whether to apply max_length//3 on each input or max_length on the concatenated input

cfg = Config()
```

</div>
<div class="column-right">

# 日本語訳

```python
# 設定を格納するためのデータクラスを定義します。
@dataclass
class Config:
    # gemma_dirは、Gemmaモデルのディレクトリのパスです。
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    # lora_dirは、LoRAモデルのチェックポイントのパスです。
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5600'
    # max_lengthは、入力シーケンスの最大長を定義します。
    max_length = 2048
    # batch_sizeは、一度に処理するデータのバッチサイズです。
    batch_size = 4
    # deviceは、モデルが使用するデバイス（GPU）を指定します。
    device = torch.device("cuda")    
    # ttaは、テスト時のデータ拡張を有効にするオプションです。
    tta = False  # test time augmentation. <prompt>-<model-b's response>-<model-a's response>
    # spread_max_lengthは、各入力にmax_length//3を適用するか、それとも連結された入力にmax_lengthを適用するかを決定します。
    spread_max_length = False  # whether to apply max_length//3 on each input or max_length on the concatenated input

# Configインスタンスを作成します。
cfg = Config()
```

</div>
</details>

In [ ]:
# 設定を格納するためのデータクラスを定義します。
@dataclass
class Config:
    # gemma_dirは、Gemmaモデルのディレクトリのパスです。
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    # lora_dirは、LoRAモデルのチェックポイントのパスです。
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5600'
    # max_lengthは、入力シーケンスの最大長を定義します。
    max_length = 2048
    # batch_sizeは、一度に処理するデータのバッチサイズです。
    batch_size = 4
    # deviceは、モデルが使用するデバイス（GPU）を指定します。
    device = torch.device("cuda")    
    # ttaは、テスト時のデータ拡張を有効にするオプションです。
    tta = False  # test time augmentation. <prompt>-<model-b's response>-<model-a's response>
    # spread_max_lengthは、各入力にmax_length//3を適用するか、それとも連結された入力にmax_lengthを適用するかを決定します。
    spread_max_length = False  # whether to apply max_length//3 on each input or max_length on the concatenated input

# Configインスタンスを作成します。
cfg = Config()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

```

</div>
<div class="column-right">

# 日本語訳

```python
# test.csvファイルを読み込み、テストデータをDataFrameとして格納します。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
```

</div>
</details>

In [ ]:
# test.csvファイルを読み込み、テストデータをDataFrameとして格納します。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

display(test.head(5))
```

</div>
<div class="column-right">

# 日本語訳

```python
# テキストを処理する関数を定義します。
# この関数は、文字列を評価し、nullを空文字列に置き換えます。
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

# テストデータフレームの'prompt'列の各テキストをprocess_text関数を使用して処理します。
test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
# 'response_a'列の各テキストも同様に処理します。
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
# 'response_b'列の各テキストも同様に処理します。
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

# 処理したテストデータの最初の5行を表示します。
display(test.head(5))
```

</div>
</details>

In [ ]:
# テキストを処理する関数を定義します。
# この関数は、文字列を評価し、nullを空文字列に置き換えます。
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

# テストデータフレームの'prompt'列の各テキストをprocess_text関数を使用して処理します。
test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
# 'response_a'列の各テキストも同様に処理します。
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
# 'response_b'列の各テキストも同様に処理します。
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

# 処理したテストデータの最初の5行を表示します。
display(test.head(5))

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = ["<prompt>: " + p for p in prompt]
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    if spread_max_length:
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    return input_ids, attention_mask
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイザーを使用してテキストをトークン化する関数を定義します。
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    # 各プロンプトに"<prompt>:"を追加します。
    prompt = ["<prompt>: " + p for p in prompt]
    # 各レスポンスに"\n\n<response_a>:"または"\n\n<response_b>:"を追加します。
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    
    # spread_max_lengthがTrueの場合、各入力に対して最大長を3で割った長さを適用します。
    if spread_max_length:
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        # 各プロンプト、レスポンスA、レスポンスBの入力IDを連結します。
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        # 入力IDに対応するアテンションマスクを生成します。
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        # プロンプト、レスポンスA、レスポンスBを連結します。
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        # トークナイザーを使用してテキストをトークン化します。
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
        
    return input_ids, attention_mask  # トークン化された入力IDとアテンションマスクを返します。
```

</div>
</details>

In [ ]:
# トークナイザーを使用してテキストをトークン化する関数を定義します。
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    # 各プロンプトに"<prompt>:"を追加します。
    prompt = ["<prompt>: " + p for p in prompt]
    # 各レスポンスに"\n\n<response_a>:"または"\n\n<response_b>:"を追加します。
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    
    # spread_max_lengthがTrueの場合、各入力に対して最大長を3で割った長さを適用します。
    if spread_max_length:
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        # 各プロンプト、レスポンスA、レスポンスBの入力IDを連結します。
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        # 入力IDに対応するアテンションマスクを生成します。
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        # プロンプト、レスポンスA、レスポンスBを連結します。
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        # トークナイザーを使用してテキストをトークン化します。
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
        
    return input_ids, attention_mask  # トークン化された入力IDとアテンションマスクを返します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%time

tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True
tokenizer.padding_side = "right"

data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
data["length"] = data["input_ids"].apply(len)

aug_data = pd.DataFrame()
aug_data["id"] = test["id"]
# swap response_a & response_b
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)
```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルの実行時間を計測します。
%%time

# GemmaTokenizerFastを指定したディレクトリからロードします。
tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
# トークナイザーにEOSトークンを追加する設定を行います。
tokenizer.add_eos_token = True
# パディングの方向を右に設定します。
tokenizer.padding_side = "right"

# 新しいデータフレームを作成します。
data = pd.DataFrame()
# 元のテストデータからIDをコピーします。
data["id"] = test["id"]
# トークナイザーを使用して、入力IDとアテンションマスクを生成します。
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
# 各入力IDの長さを計算して新しい列に追加します。
data["length"] = data["input_ids"].apply(len)

# 拡張データのための新しいデータフレームを作成します。
aug_data = pd.DataFrame()
# 元のテストデータからIDをコピーします。
aug_data["id"] = test["id"]
# response_aとresponse_bを入れ替えたトークナイゼーションを実行します。
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
# 各入力IDの長さを計算して新しい列に追加します。
aug_data["length"] = aug_data["input_ids"].apply(len)
```

</div>
</details>

In [ ]:
# このセルの実行時間を計測します。
%%time

# GemmaTokenizerFastを指定したディレクトリからロードします。
tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
# トークナイザーにEOSトークンを追加する設定を行います。
tokenizer.add_eos_token = True
# パディングの方向を右に設定します。
tokenizer.padding_side = "right"

# 新しいデータフレームを作成します。
data = pd.DataFrame()
# 元のテストデータからIDをコピーします。
data["id"] = test["id"]
# トークナイザーを使用して、入力IDとアテンションマスクを生成します。
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
# 各入力IDの長さを計算して新しい列に追加します。
data["length"] = data["input_ids"].apply(len)

# 拡張データのための新しいデータフレームを作成します。
aug_data = pd.DataFrame()
# 元のテストデータからIDをコピーします。
aug_data["id"] = test["id"]
# response_aとresponse_bを入れ替えたトークナイゼーションを実行します。
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
# 各入力IDの長さを計算して新しい列に追加します。
aug_data["length"] = aug_data["input_ids"].apply(len)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Load base model on GPU 0
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False,
)

# Load base model on GPU 1
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,
    use_cache=False,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# GPU 0にベースモデルをロードします。
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,  # モデルをGPU 0にマッピングします。
    use_cache=False,  # キャッシュを使用しない設定です。
)

# GPU 1にベースモデルをロードします。
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,  # モデルをGPU 1にマッピングします。
    use_cache=False,  # キャッシュを使用しない設定です。
)
```

</div>
</details>

In [ ]:
# GPU 0にベースモデルをロードします。
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,  # モデルをGPU 0にマッピングします。
    use_cache=False,  # キャッシュを使用しない設定です。
)

# GPU 1にベースモデルをロードします。
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,  # モデルをGPU 1にマッピングします。
    use_cache=False,  # キャッシュを使用しない設定です。
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)
```

</div>
<div class="column-right">

# 日本語訳

```python
# LoRAモデルをベースモデルに適用します。
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)  # GPU 0のモデルにLoRA設定を適用します。
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)  # GPU 1のモデルにLoRA設定を適用します。
```

</div>
</details>

In [ ]:
# LoRAモデルをベースモデルに適用します。
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)  # GPU 0のモデルにLoRA設定を適用します。
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)  # GPU 1のモデルにLoRA設定を適用します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()
        
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df
```

</div>
<div class="column-right">

# 日本語訳

```python
# 推論を行う関数を定義します。この関数は、指定されたデータフレームに基づいてモデルに対して推論を行います。
@torch.no_grad()  # 勾配計算を無効にします。推論時には不要です。
@torch.cuda.amp.autocast()  # 自動混合精度を使用して、計算を効率化します。
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各モデルの勝率を格納するリストを初期化します。
    
    # データフレームをバッチ処理で処理します。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終了インデックスを決定します。
        tmp = df.iloc[start_idx:end_idx]  # 現在のバッチを取得します。
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストに変換します。
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクをリストに変換します。

        # 入力をパディングしてテンソル形式に変換します。
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        
        # モデルに対して推論を実行します。
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()  # 出力のロジットをソフトマックス関数で確率に変換します。
        
        # 各モデルの勝率をリストに追加します。
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    # データフレームに勝率の結果を追加します。
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df  # 更新されたデータフレームを返します。
```

</div>
</details>

In [ ]:
# 推論を行う関数を定義します。この関数は、指定されたデータフレームに基づいてモデルに対して推論を行います。
@torch.no_grad()  # 勾配計算を無効にします。推論時には不要です。
@torch.cuda.amp.autocast()  # 自動混合精度を使用して、計算を効率化します。
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各モデルの勝率を格納するリストを初期化します。
    
    # データフレームをバッチ処理で処理します。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終了インデックスを決定します。
        tmp = df.iloc[start_idx:end_idx]  # 現在のバッチを取得します。
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストに変換します。
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクをリストに変換します。

        # 入力をパディングしてテンソル形式に変換します。
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        
        # モデルに対して推論を実行します。
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()  # 出力のロジットをソフトマックス関数で確率に変換します。
        
        # 各モデルの勝率をリストに追加します。
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    # データフレームに勝率の結果を追加します。
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df  # 更新されたデータフレームを返します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

# sort by input length to fully leverage dynaminc padding
data = data.sort_values("length", ascending=False)
# the total #tokens in sub_1 and sub_2 should be more or less the same
sub_1 = data.iloc[0::2].copy()
sub_2 = data.iloc[1::2].copy()

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

print(f"elapsed time: {time.time() - st}")
```

</div>
<div class="column-right">

# 日本語訳

```python
# 処理の開始時間を記録します。
st = time.time()

# 入力の長さでソートし、動的パディングを最大限に活用します。
data = data.sort_values("length", ascending=False)

# データを2つのサブセットに分割します。これにより、トークン数がほぼ同じになることを期待します。
sub_1 = data.iloc[0::2].copy()  # 偶数のインデックスを持つ行をサブセット1にコピーします。
sub_2 = data.iloc[1::2].copy()  # 奇数のインデックスを持つ行をサブセット2にコピーします。

# スレッドプールを使用して並行処理を実行します。
with ThreadPoolExecutor(max_workers=2) as executor:
    # inference関数をサブセット1とサブセット2のそれぞれにモデルを適用して実行します。
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

# 結果を結合して新しいデータフレームを作成します。
result_df = pd.concat(list(results), axis=0)

# 勝率の確率を取得します。
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

# 処理が完了するまでの経過時間を表示します。
print(f"elapsed time: {time.time() - st}")
```

</div>
</details>

In [ ]:
# 処理の開始時間を記録します。
st = time.time()

# 入力の長さでソートし、動的パディングを最大限に活用します。
data = data.sort_values("length", ascending=False)

# データを2つのサブセットに分割します。これにより、トークン数がほぼ同じになることを期待します。
sub_1 = data.iloc[0::2].copy()  # 偶数のインデックスを持つ行をサブセット1にコピーします。
sub_2 = data.iloc[1::2].copy()  # 奇数のインデックスを持つ行をサブセット2にコピーします。

# スレッドプールを使用して並行処理を実行します。
with ThreadPoolExecutor(max_workers=2) as executor:
    # inference関数をサブセット1とサブセット2のそれぞれにモデルを適用して実行します。
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

# 結果を結合して新しいデータフレームを作成します。
result_df = pd.concat(list(results), axis=0)

# 勝率の確率を取得します。
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

# 処理が完了するまでの経過時間を表示します。
print(f"elapsed time: {time.time() - st}")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

if cfg.tta:
    data = aug_data.sort_values("length", ascending=False)  # sort by input length to boost speed
    sub_1 = data.iloc[0::2].copy()
    sub_2 = data.iloc[1::2].copy()

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    tta_result_df = pd.concat(list(results), axis=0)
    # recall TTA's order is flipped
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # average original result and TTA result.
    proba = (proba + tta_proba) / 2

print(f"elapsed time: {time.time() - st}")
```

</div>
<div class="column-right">

# 日本語訳

```python
# 処理の開始時間を記録します。
st = time.time()

# テスト時データ拡張（TTA）が有効な場合の処理を行います。
if cfg.tta:
    # 入力の長さでソートし、処理速度を向上させます。
    data = aug_data.sort_values("length", ascending=False)
    # データを2つのサブセットに分割します。
    sub_1 = data.iloc[0::2].copy()
    sub_2 = data.iloc[1::2].copy()

    # スレッドプールを使用して並行処理を実行します。
    with ThreadPoolExecutor(max_workers=2) as executor:
        # inference関数をサブセット1とサブセット2のそれぞれにモデルを適用して実行します。
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    # 結果を結合して新しいデータフレームを作成します。
    tta_result_df = pd.concat(list(results), axis=0)
    # TTAの結果の順序が反転しているため、winner_model_bを先にし、winner_model_aを後にします。
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # 元の結果とTTAの結果を平均します。
    proba = (proba + tta_proba) / 2

# 処理が完了するまでの経過時間を表示します。
print(f"elapsed time: {time.time() - st}")
```

</div>
</details>

In [ ]:
# 処理の開始時間を記録します。
st = time.time()

# テスト時データ拡張（TTA）が有効な場合の処理を行います。
if cfg.tta:
    # 入力の長さでソートし、処理速度を向上させます。
    data = aug_data.sort_values("length", ascending=False)
    # データを2つのサブセットに分割します。
    sub_1 = data.iloc[0::2].copy()
    sub_2 = data.iloc[1::2].copy()

    # スレッドプールを使用して並行処理を実行します。
    with ThreadPoolExecutor(max_workers=2) as executor:
        # inference関数をサブセット1とサブセット2のそれぞれにモデルを適用して実行します。
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    # 結果を結合して新しいデータフレームを作成します。
    tta_result_df = pd.concat(list(results), axis=0)
    # TTAの結果の順序が反転しているため、winner_model_bを先にし、winner_model_aを後にします。
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # 元の結果とTTAの結果を平均します。
    proba = (proba + tta_proba) / 2

# 処理が完了するまでの経過時間を表示します。
print(f"elapsed time: {time.time() - st}")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
result_df.loc[:, "winner_model_a"] = proba[:, 0]
result_df.loc[:, "winner_model_b"] = proba[:, 1]
result_df.loc[:, "winner_tie"] = proba[:, 2]
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 結果データフレームに勝率の確率を追加します。
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝率を設定します。
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝率を設定します。
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 引き分けの勝率を設定します。

# 提出用のデータフレームを作成します。
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
# データフレームをCSVファイルにエクスポートします。
submission_df.to_csv('submission.csv', index=False)

# 提出用データフレームの内容を表示します。
display(submission_df)
```

</div>
</details>

In [ ]:
# 結果データフレームに勝率の確率を追加します。
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝率を設定します。
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝率を設定します。
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 引き分けの勝率を設定します。

# 提出用のデータフレームを作成します。
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
# データフレームをCSVファイルにエクスポートします。
submission_df.to_csv('submission.csv', index=False)

# 提出用データフレームの内容を表示します。
display(submission_df)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Importing Libraries

</div>
<div class="column-right">

# 日本語訳

## ライブラリのインポート

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータとサンプル提出ファイルをCSV形式で読み込みます。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込みます。
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # サンプル提出ファイルを読み込みます。
```

</div>
</details>

In [ ]:
# テストデータとサンプル提出ファイルをCSV形式で読み込みます。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込みます。
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # サンプル提出ファイルを読み込みます。

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Tokenize

## Load model 
> We load 1 model on each gpu.  

## Inference

</div>
<div class="column-right">

# 日本語訳

## トークナイズ

## モデルの読み込み 
> 各GPUに1つのモデルを読み込みます。  

## 推論

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

sample_sub[TARGETS] = result_df[TARGETS]
```

</div>
<div class="column-right">

# 日本語訳

```python
# サンプル提出データフレームに結果データフレームから勝率の確率を追加します。
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']  # 対象列を定義します。

# 結果データフレームの対象列をサンプル提出データフレームにコピーします。
sample_sub[TARGETS] = result_df[TARGETS]
```

</div>
</details>

In [ ]:
# サンプル提出データフレームに結果データフレームから勝率の確率を追加します。
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']  # 対象列を定義します。

# 結果データフレームの対象列をサンプル提出データフレームにコピーします。
sample_sub[TARGETS] = result_df[TARGETS]

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
llama_preds = result_df[TARGETS].values
```

</div>
<div class="column-right">

# 日本語訳

```python
# 結果データフレームから対象列の予測をNumPy配列として取得します。
llama_preds = result_df[TARGETS].values  # モデルの予測結果を配列に格納します。
```

</div>
</details>

In [ ]:
# 結果データフレームから対象列の予測をNumPy配列として取得します。
llama_preds = result_df[TARGETS].values  # モデルの予測結果を配列に格納します。

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## LGBM + tfidf

</div>
<div class="column-right">

# 日本語訳

## LGBM + TF-IDF

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
TAG = 'lmsys-chatbot-arena'
RUNPOD = os.path.exists('/workspace/')
KAGGLE = not RUNPOD
if KAGGLE: 
    print('kaggle')
```

</div>
<div class="column-right">

# 日本語訳

```python
# 実行環境を確認します。
TAG = 'lmsys-chatbot-arena'  # タグを定義します。
RUNPOD = os.path.exists('/workspace/')  # 実行環境がRunPodかどうかを確認します。
KAGGLE = not RUNPOD  # Kaggleで実行しているかどうかを判断します。

# Kaggleで実行している場合、メッセージを表示します。
if KAGGLE: 
    print('kaggle')  # Kaggle環境であることを表示します。
```

</div>
</details>

In [ ]:
# 実行環境を確認します。
TAG = 'lmsys-chatbot-arena'  # タグを定義します。
RUNPOD = os.path.exists('/workspace/')  # 実行環境がRunPodかどうかを確認します。
KAGGLE = not RUNPOD  # Kaggleで実行しているかどうかを判断します。

# Kaggleで実行している場合、メッセージを表示します。
if KAGGLE: 
    print('kaggle')  # Kaggle環境であることを表示します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
try:
    import pandas as pd
except:
    !pip install -q kaggle
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm 
    !pip install -q protobuf 
    !pip install -q numba
```

</div>
<div class="column-right">

# 日本語訳

```python
# pandasライブラリをインポートします。失敗した場合は必要なライブラリをインストールします。
try:
    import pandas as pd
except:
    # Kaggle環境に必要なパッケージをインストールします。
    !pip install -q kaggle  # Kaggle APIをインストールします。
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm  # データ処理と可視化のためのライブラリをインストールします。
    !pip install -q protobuf  # Protobufライブラリをインストールします。
    !pip install -q numba  # Numbaライブラリをインストールします。
```

</div>
</details>

In [ ]:
# pandasライブラリをインポートします。失敗した場合は必要なライブラリをインストールします。
try:
    import pandas as pd
except:
    # Kaggle環境に必要なパッケージをインストールします。
    !pip install -q kaggle  # Kaggle APIをインストールします。
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm  # データ処理と可視化のためのライブラリをインストールします。
    !pip install -q protobuf  # Protobufライブラリをインストールします。
    !pip install -q numba  # Numbaライブラリをインストールします。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)

if RUNPOD:
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json
        !chmod 600 /root/.kaggle/kaggle.json

    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/ 
        
    if not os.path.exists('/data/'):
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')    
```

</div>
<div class="column-right">

# 日本語訳

```python
# データのパスを設定します。実行環境に応じて異なるパスを指定します。
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)  # Kaggle環境であれば、競技用のデータパスを指定します。

# RunPod環境で実行している場合の設定を行います。
if RUNPOD:
    # Kaggle APIの設定ファイルが存在しない場合、作成しコピーします。
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle  # Kaggle用のディレクトリを作成します。
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json  # Kaggle APIの設定ファイルをコピーします。
        !chmod 600 /root/.kaggle/kaggle.json  # アクセス権を設定します。

    # データがまだダウンロードされていない場合、Kaggleからコンペティションデータをダウンロードします。
    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/  # Kaggleのコンペティションデータをダウンロードします。
        
    # データディレクトリが存在しない場合、ZIPファイルを解凍します。
    if not os.path.exists('/data/'):
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')  # ZIPファイルを解凍します。
```

</div>
</details>

In [ ]:
# データのパスを設定します。実行環境に応じて異なるパスを指定します。
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)  # Kaggle環境であれば、競技用のデータパスを指定します。

# RunPod環境で実行している場合の設定を行います。
if RUNPOD:
    # Kaggle APIの設定ファイルが存在しない場合、作成しコピーします。
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle  # Kaggle用のディレクトリを作成します。
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json  # Kaggle APIの設定ファイルをコピーします。
        !chmod 600 /root/.kaggle/kaggle.json  # アクセス権を設定します。

    # データがまだダウンロードされていない場合、Kaggleからコンペティションデータをダウンロードします。
    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/  # Kaggleのコンペティションデータをダウンロードします。
        
    # データディレクトリが存在しない場合、ZIPファイルを解凍します。
    if not os.path.exists('/data/'):
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')  # ZIPファイルを解凍します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
INPUT_PATH = '/kaggle/input/'  
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'
print(MODEL_PATH)

CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'
SAVE_PATH = MODEL_PATH if not KAGGLE else ''
```

</div>
<div class="column-right">

# 日本語訳

```python
# 入力データとモデルのパスを設定します。
INPUT_PATH = '/kaggle/input/'  # Kaggleの入力データパス
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'  # モデルとロジットの保存先

# Kaggle環境の場合、モデルのパスを設定します。lsys-modelsを含むディレクトリを探索します。
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'
print(MODEL_PATH)  # モデルのパスを表示します。

# コードの保存先を設定します。
CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'
# 保存先を設定します。
SAVE_PATH = MODEL_PATH if not KAGGLE else ''  # Kaggle環境では保存先を空に設定します。
```

</div>
</details>

In [ ]:
# 入力データとモデルのパスを設定します。
INPUT_PATH = '/kaggle/input/'  # Kaggleの入力データパス
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'  # モデルとロジットの保存先

# Kaggle環境の場合、モデルのパスを設定します。lsys-modelsを含むディレクトリを探索します。
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'
print(MODEL_PATH)  # モデルのパスを表示します。

# コードの保存先を設定します。
CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'
# 保存先を設定します。
SAVE_PATH = MODEL_PATH if not KAGGLE else ''  # Kaggle環境では保存先を空に設定します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイザーの並列処理を無効にするための環境変数を設定します。
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
```

</div>
</details>

In [ ]:
# トークナイザーの並列処理を無効にするための環境変数を設定します。
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train = pd.read_csv(open(DATA + 'train.csv', 'r'))
test = pd.read_csv(open(DATA + 'test.csv', 'r'))
sample = pd.read_csv(DATA + 'sample_submission.csv')
print(len(train), len(test))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 訓練データ、テストデータ、サンプル提出データをCSV形式で読み込みます。
train = pd.read_csv(open(DATA + 'train.csv', 'r'))  # 訓練データを読み込みます。
test = pd.read_csv(open(DATA + 'test.csv', 'r'))    # テストデータを読み込みます。
sample = pd.read_csv(DATA + 'sample_submission.csv')  # サンプル提出データを読み込みます。

# 訓練データとテストデータの行数を表示します。
print(len(train), len(test))  # 訓練データとテストデータの長さを表示します。
```

</div>
</details>

In [ ]:
# 訓練データ、テストデータ、サンプル提出データをCSV形式で読み込みます。
train = pd.read_csv(open(DATA + 'train.csv', 'r'))  # 訓練データを読み込みます。
test = pd.read_csv(open(DATA + 'test.csv', 'r'))    # テストデータを読み込みます。
sample = pd.read_csv(DATA + 'sample_submission.csv')  # サンプル提出データを読み込みます。

# 訓練データとテストデータの行数を表示します。
print(len(train), len(test))  # 訓練データとテストデータの長さを表示します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
params = {}
if False: 
    pass;
    params['subsample'] = 30
else:
    params['fold'] = -1


params['n_epochs'] = 1
params['n_lgb'] = 1
params['model'] = 'microsoft/deberta-v3-small'
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルのパラメータを設定します。
params = {}
# 条件に応じてパラメータを設定します。このブロックは実行されないため、何も設定しません。
if False: 
    pass;
    params['subsample'] = 30
else:
    params['fold'] = -1  # foldの値を-1に設定します。

# モデルの訓練に関するその他のパラメータを設定します。
params['n_epochs'] = 1  # 訓練のエポック数を1に設定します。
params['n_lgb'] = 1  # LightGBMのモデル数を1に設定します。
params['model'] = 'microsoft/deberta-v3-small'  # 使用するモデルをDeBERTaに設定します。
```

</div>
</details>

In [ ]:
# モデルのパラメータを設定します。
params = {}
# 条件に応じてパラメータを設定します。このブロックは実行されないため、何も設定しません。
if False: 
    pass;
    params['subsample'] = 30
else:
    params['fold'] = -1  # foldの値を-1に設定します。

# モデルの訓練に関するその他のパラメータを設定します。
params['n_epochs'] = 1  # 訓練のエポック数を1に設定します。
params['n_lgb'] = 1  # LightGBMのモデル数を1に設定します。
params['model'] = 'microsoft/deberta-v3-small'  # 使用するモデルをDeBERTaに設定します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# params = {}
FULL = params.get('fold', 0) < 0
N_FOLDS = int(params.get('n_folds', 3)); 
FOLD = int(params.get('fold', 0))
SEED = int(params.get('seed', 3))
SS = int(params.get('subsample', 1))

print(N_FOLDS, FOLD, SEED, SS)
```

</div>
<div class="column-right">

# 日本語訳

```python
# パラメータに基づいて各種設定を行います。
# foldが0未満の場合、FULLをTrueに設定します。
FULL = params.get('fold', 0) < 0
# n_foldsの数を取得します。デフォルトは3です。
N_FOLDS = int(params.get('n_folds', 3))  
# foldの値を取得します。デフォルトは0です。
FOLD = int(params.get('fold', 0))
# seed値を取得します。デフォルトは3です。
SEED = int(params.get('seed', 3))
# subsample値を取得します。デフォルトは1です。
SS = int(params.get('subsample', 1))

# 設定した値を表示します。
print(N_FOLDS, FOLD, SEED, SS)  # N_FOLDS、FOLD、SEED、SSの値を表示します。
```

</div>
</details>

In [ ]:
# パラメータに基づいて各種設定を行います。
# foldが0未満の場合、FULLをTrueに設定します。
FULL = params.get('fold', 0) < 0
# n_foldsの数を取得します。デフォルトは3です。
N_FOLDS = int(params.get('n_folds', 3))  
# foldの値を取得します。デフォルトは0です。
FOLD = int(params.get('fold', 0))
# seed値を取得します。デフォルトは3です。
SEED = int(params.get('seed', 3))
# subsample値を取得します。デフォルトは1です。
SS = int(params.get('subsample', 1))

# 設定した値を表示します。
print(N_FOLDS, FOLD, SEED, SS)  # N_FOLDS、FOLD、SEED、SSの値を表示します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from sklearn.model_selection import StratifiedKFold

def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state = SEED, shuffle = True)\
                    .split(X = np.zeros(len(train)), y = train.iloc[:, -3:].idxmax(1)))

train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
if SS > 1:
    train_ids, test_ids = train_ids[::SS], test_ids[::SS]

print(len(train_ids), len(test_ids));  assert set(train_ids) & set(test_ids) == set() 
```

</div>
<div class="column-right">

# 日本語訳

```python
# StratifiedKFoldをインポートします。これは層化K-Foldクロスバリデーションを実施するためのクラスです。
from sklearn.model_selection import StratifiedKFold

# K-Foldのインデックスを取得する関数を定義します。
def get_folds(train): 
    # StratifiedKFoldを使用して、訓練データの分割を行い、インデックスを返します。
    return list(StratifiedKFold(N_FOLDS, random_state=SEED, shuffle=True)\
                    .split(X=np.zeros(len(train)), y=train.iloc[:, -3:].idxmax(1)))  # 最後の3列のインデックスが最大のものをラベルとして使用します。

# 指定されたFOLDまたはFULLを考慮して、訓練データとテストデータのIDを取得します。
train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
# subsampleが1より大きい場合、IDを間引いたリストにします。
if SS > 1:
    train_ids, test_ids = train_ids[::SS], test_ids[::SS]

# 訓練データとテストデータのIDの長さを表示します。
print(len(train_ids), len(test_ids))
# 訓練データとテストデータのIDで重複がないことを確認します。
assert set(train_ids) & set(test_ids) == set()  # 重複がある場合、エラーを出力します。
```

</div>
</details>

In [ ]:
# StratifiedKFoldをインポートします。これは層化K-Foldクロスバリデーションを実施するためのクラスです。
from sklearn.model_selection import StratifiedKFold

# K-Foldのインデックスを取得する関数を定義します。
def get_folds(train): 
    # StratifiedKFoldを使用して、訓練データの分割を行い、インデックスを返します。
    return list(StratifiedKFold(N_FOLDS, random_state=SEED, shuffle=True)\
                    .split(X=np.zeros(len(train)), y=train.iloc[:, -3:].idxmax(1)))  # 最後の3列のインデックスが最大のものをラベルとして使用します。

# 指定されたFOLDまたはFULLを考慮して、訓練データとテストデータのIDを取得します。
train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
# subsampleが1より大きい場合、IDを間引いたリストにします。
if SS > 1:
    train_ids, test_ids = train_ids[::SS], test_ids[::SS]

# 訓練データとテストデータのIDの長さを表示します。
print(len(train_ids), len(test_ids))
# 訓練データとテストデータのIDで重複がないことを確認します。
assert set(train_ids) & set(test_ids) == set()  # 重複がある場合、エラーを出力します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
torch.manual_seed(datetime.datetime.now().microsecond)
random.seed(datetime.datetime.now().microsecond)
np.random.seed(datetime.datetime.now().microsecond)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 乱数のシードを設定します。これにより、毎回の実行が再現可能になります。
torch.manual_seed(datetime.datetime.now().microsecond)  # PyTorchのシードを設定します。
random.seed(datetime.datetime.now().microsecond)  # Pythonのrandomモジュールのシードを設定します。
np.random.seed(datetime.datetime.now().microsecond)  # NumPyのシードを設定します。
```

</div>
</details>

In [ ]:
# 乱数のシードを設定します。これにより、毎回の実行が再現可能になります。
torch.manual_seed(datetime.datetime.now().microsecond)  # PyTorchのシードを設定します。
random.seed(datetime.datetime.now().microsecond)  # Pythonのrandomモジュールのシードを設定します。
np.random.seed(datetime.datetime.now().microsecond)  # NumPyのシードを設定します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
TRAIN = False
INFER = True 
SAVE = False
```

</div>
<div class="column-right">

# 日本語訳

```python
# モードの設定を行います。
TRAIN = False  # 訓練モードを無効にします。
INFER = True   # 推論モードを有効にします。
SAVE = False   # 保存モードを無効にします。
```

</div>
</details>

In [ ]:
# モードの設定を行います。
TRAIN = False  # 訓練モードを無効にします。
INFER = True   # 推論モードを有効にします。
SAVE = False   # 保存モードを無効にします。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
```

</div>
<div class="column-right">

# 日本語訳

```python
# LightGBMライブラリと、テキストデータの前処理を行うためのCountVectorizerをインポートします。
import lightgbm as lgb  # LightGBMライブラリをインポートします。これは高速な決定木アルゴリズムです。
from sklearn.feature_extraction.text import CountVectorizer  # テキストデータをベクトル化するためのクラスをインポートします。
```

</div>
</details>

In [ ]:
# LightGBMライブラリと、テキストデータの前処理を行うためのCountVectorizerをインポートします。
import lightgbm as lgb  # LightGBMライブラリをインポートします。これは高速な決定木アルゴリズムです。
from sklearn.feature_extraction.text import CountVectorizer  # テキストデータをベクトル化するためのクラスをインポートします。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
LGB = True
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0
INFER_LGB = not TRAIN and LGB
```

</div>
<div class="column-right">

# 日本語訳

```python
# LightGBMの利用設定を行います。
LGB = True  # LightGBMを使用する設定をTrueにします。
# 訓練時のLightGBMの実行設定を決定します。
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0  # 訓練中にLightGBMを実行すべきかどうかを決定します。
# 推論時のLightGBMの実行設定を決定します。
INFER_LGB = not TRAIN and LGB  # 訓練中でない場合にLightGBMを使用する設定を決定します。
```

</div>
</details>

In [ ]:
# LightGBMの利用設定を行います。
LGB = True  # LightGBMを使用する設定をTrueにします。
# 訓練時のLightGBMの実行設定を決定します。
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0  # 訓練中にLightGBMを実行すべきかどうかを決定します。
# 推論時のLightGBMの実行設定を決定します。
INFER_LGB = not TRAIN and LGB  # 訓練中でない場合にLightGBMを使用する設定を決定します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 学習済みのCountVectorizerオブジェクトをファイルから読み込みます。
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))  # CountVectorizerを読み込みます。
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))  # 別のCountVectorizerを読み込みます。
```

</div>
</details>

In [ ]:
# 学習済みのCountVectorizerオブジェクトをファイルから読み込みます。
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))  # CountVectorizerを読み込みます。
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))  # 別のCountVectorizerを読み込みます。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def symlog(x):
    return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)

def dense(x):
    x = np.asarray(x.astype(np.float32).todense())
    x = symlog(x)
    return x

def get_features(df):
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    v = np.hstack([
          afeat - bfeat, np.abs(afeat - bfeat), 
        ])
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)
    except:
        pass

    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)
            
    extras.append(df[c].str.len())
    extras.append(df[c].str.split().apply(lambda x: len(x)))
    
    extras = np.stack(extras, axis = 1)
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])
    return np.hstack([v, extras])
```

</div>
<div class="column-right">

# 日本語訳

```python
# シンメトリックログ変換を行う関数を定義します。
def symlog(x):
    return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)  # xのシンメトリックログを計算します。

# 疎行列をDense行列に変換し、シンメトリックログ変換を適用する関数を定義します。
def dense(x):
    x = np.asarray(x.astype(np.float32).todense())  # 疎行列をDense行列に変換します。
    x = symlog(x)  # シンメトリックログ変換を適用します。
    return x

# 特徴量を取得するための関数を定義します。
def get_features(df):
    # プロンプトから生成される特徴量
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])  # CountVectorizerを使用して特徴量を抽出します。

    # レスポンスAから生成される特徴量
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    
    # レスポンスBから生成される特徴量
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    # AとBのレスポンスの比較特徴量を作成します。
    v = np.hstack([
          afeat - bfeat, np.abs(afeat - bfeat),  # レスポンスAとレスポンスBの差分を含む特徴量
        ])
    
    # モデル数によってvを調整します。
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)
    except:
        pass

    # 追加の特徴量を生成します。
    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']  # 特徴量を生成するための追加項目
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)  # 特定の文字の出現回数をカウントします。
            
    extras.append(df[c].str.len())  # 各列の文字数を追加します。
    extras.append(df[c].str.split().apply(lambda x: len(x)))  # 各列の単語数を追加します。
    
    extras = np.stack(extras, axis=1)  # 追加特徴量を1つの配列にまとめます。
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])  # 追加特徴量を変換します。
    
    return np.hstack([v, extras])  # 最後に全ての特徴量を結合して返します。
```

</div>
</details>

In [ ]:
# シンメトリックログ変換を行う関数を定義します。
def symlog(x):
    return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)  # xのシンメトリックログを計算します。

# 疎行列をDense行列に変換し、シンメトリックログ変換を適用する関数を定義します。
def dense(x):
    x = np.asarray(x.astype(np.float32).todense())  # 疎行列をDense行列に変換します。
    x = symlog(x)  # シンメトリックログ変換を適用します。
    return x

# 特徴量を取得するための関数を定義します。
def get_features(df):
    # プロンプトから生成される特徴量
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])  # CountVectorizerを使用して特徴量を抽出します。

    # レスポンスAから生成される特徴量
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    
    # レスポンスBから生成される特徴量
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    # AとBのレスポンスの比較特徴量を作成します。
    v = np.hstack([
          afeat - bfeat, np.abs(afeat - bfeat),  # レスポンスAとレスポンスBの差分を含む特徴量
        ])
    
    # モデル数によってvを調整します。
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)
    except:
        pass

    # 追加の特徴量を生成します。
    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']  # 特徴量を生成するための追加項目
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)  # 特定の文字の出現回数をカウントします。
            
    extras.append(df[c].str.len())  # 各列の文字数を追加します。
    extras.append(df[c].str.split().apply(lambda x: len(x)))  # 各列の単語数を追加します。
    
    extras = np.stack(extras, axis=1)  # 追加特徴量を1つの配列にまとめます。
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])  # 追加特徴量を変換します。
    
    return np.hstack([v, extras])  # 最後に全ての特徴量を結合して返します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 学習済みのLightGBMモデルをファイルから読み込みます。
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))  # LightGBMモデルを読み込みます。
```

</div>
</details>

In [ ]:
# 学習済みのLightGBMモデルをファイルから読み込みます。
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))  # LightGBMモデルを読み込みます。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
if INFER and params.get('n_lgb', 1) > 0:
    df = test
    yps = []; b = 1000
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])
        ypms = []
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))
        yps.append(np.stack(ypms).mean(0))
        print('.', end = '')
        
        if len(yps) % 2 == 0:
            gc.collect()
    print()

    yp = np.concatenate(yps)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 推論が有効で、LightGBMモデルの数が0より大きい場合の処理を行います。
if INFER and params.get('n_lgb', 1) > 0:
    df = test  # テストデータフレームをdfに設定します。
    yps = []  # 複数の予測結果を格納するリストを初期化します。
    b = 1000  # 一度に処理するデータのバッチサイズを設定します。
    
    # データをバッチ処理で推論します。
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])  # 特徴量を取得します。
        ypms = []  # 各モデルの予測結果を格納するリストを初期化します。
        
        # 全てのLightGBMモデルに対して予測を行います。
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))  # モデルからの予測確率を取得します。
        
        # 各モデルの予測確率を平均化してypsに追加します。
        yps.append(np.stack(ypms).mean(0))
        print('.', end = '')  # 進行状況を表示します。
        
        # 2バッチごとにガベージコレクションを実行します。
        if len(yps) % 2 == 0:
            gc.collect()
    print()  # 改行します。

    # すべての予測結果を連結します。
    yp = np.concatenate(yps)  # 最終的な予測結果を取得します。
```

</div>
</details>

In [ ]:
# 推論が有効で、LightGBMモデルの数が0より大きい場合の処理を行います。
if INFER and params.get('n_lgb', 1) > 0:
    df = test  # テストデータフレームをdfに設定します。
    yps = []  # 複数の予測結果を格納するリストを初期化します。
    b = 1000  # 一度に処理するデータのバッチサイズを設定します。
    
    # データをバッチ処理で推論します。
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])  # 特徴量を取得します。
        ypms = []  # 各モデルの予測結果を格納するリストを初期化します。
        
        # 全てのLightGBMモデルに対して予測を行います。
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))  # モデルからの予測確率を取得します。
        
        # 各モデルの予測確率を平均化してypsに追加します。
        yps.append(np.stack(ypms).mean(0))
        print('.', end = '')  # 進行状況を表示します。
        
        # 2バッチごとにガベージコレクションを実行します。
        if len(yps) % 2 == 0:
            gc.collect()
    print()  # 改行します。

    # すべての予測結果を連結します。
    yp = np.concatenate(yps)  # 最終的な予測結果を取得します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_preds = yp
```

</div>
<div class="column-right">

# 日本語訳

```python
# LightGBMモデルからの最終的な予測結果をlgb_predsに格納します。
lgb_preds = yp  # 推論結果をlgb_predsに設定します。
```

</div>
</details>

In [ ]:
# LightGBMモデルからの最終的な予測結果をlgb_predsに格納します。
lgb_preds = yp  # 推論結果をlgb_predsに設定します。

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Blend predictions

$\operatorname{preds} = 0.12 \cdot \operatorname{lgbm boosting preds} + 0.8 \cdot \operatorname{llama preds}$


</div>
<div class="column-right">

# 日本語訳

## 予測のブレンド

$\operatorname{preds} = 0.12 \cdot \operatorname{LGBMブースティング予測} + 0.8 \cdot \operatorname{Llama予測}$

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_wt = 0.6
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds
```

</div>
<div class="column-right">

# 日本語訳

```python
# LGBMの重みを設定し、予測をブレンドします。
lgb_wt = 0.6  # LGBMモデルの重みを設定します。
# LGBM予測とLlama予測を重み付けして加算し、最終的な予測を得ます。
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds  # 予測のブレンドを実行します。
```

</div>
</details>

In [ ]:
# LGBMの重みを設定し、予測をブレンドします。
lgb_wt = 0.6  # LGBMモデルの重みを設定します。
# LGBM予測とLlama予測を重み付けして加算し、最終的な予測を得ます。
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds  # 予測のブレンドを実行します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
out = pd.DataFrame(preds, index=df.id, columns=train.columns[-3:])
display(out.head())
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最終的な予測結果をDataFrameとして作成します。
out = pd.DataFrame(preds, index=df.id, columns=train.columns[-3:])  # 予測値をDataFrameに格納します。
# 最初の数行を表示します。
display(out.head())  # 生成したDataFrameの先頭を表示します。
```

</div>
</details>

In [ ]:
# 最終的な予測結果をDataFrameとして作成します。
out = pd.DataFrame(preds, index=df.id, columns=train.columns[-3:])  # 予測値をDataFrameに格納します。
# 最初の数行を表示します。
display(out.head())  # 生成したDataFrameの先頭を表示します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
out.to_csv('submission.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最終的な予測結果をCSVファイルとして保存します。
out.to_csv('submission.csv')  # 生成したDataFrameを'submission.csv'として出力します。
```

</div>
</details>

In [ ]:
# 最終的な予測結果をCSVファイルとして保存します。
out.to_csv('submission.csv')  # 生成したDataFrameを'submission.csv'として出力します。